# Set up analysis

In [14]:

# Load modules
import datacube
import fiona
import glob
import itertools
import os
import rasterio
import rasterio.features
import xarray as xr
import numpy as np
from collections import OrderedDict
from datacube.helpers import ga_pq_fuser
from datacube.storage import masking
from datacube.storage.masking import mask_invalid_data
from datacube.utils import geometry
from datacube.utils.geometry import CRS
from matplotlib import pyplot as plt
from osgeo import gdal
from shapely.geometry import Point
from shapely.geometry import shape
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz

# Set up datacube instance
dc = datacube.Datacube(app = 'Random forest classification')


def load_nbart(sensor, query):
    
    '''
    Loads nbart data for a sensor, masks using pq, then filters out terrain -999s
    function written 23-08-2017 based on dc v1.5.1
    '''  
    
    product_name = '{}_{}_albers'.format(sensor, 'nbart')
    print('Loading {}'.format(product_name))
    ds = dc.load(product = product_name,
                 group_by = 'solar_day', 
                 **query)
    
    if ds:       
   
        print('Loaded {}'.format(product_name))
        
        # Extract PQ data for masking
        mask_product = '{}_{}_albers'.format(sensor, 'pq')
        sensor_pq = dc.load(product = mask_product, 
                            fuse_func = ga_pq_fuser,
                            group_by = 'solar_day', 
                            **query)
        
        if sensor_pq:
            
            print('Making mask {}'.format(mask_product))
            cloud_free = masking.make_mask(sensor_pq.pixelquality,
                                           cloud_acca = 'no_cloud',
                                           cloud_shadow_acca = 'no_cloud_shadow',                           
                                           cloud_shadow_fmask = 'no_cloud_shadow',
                                           cloud_fmask = 'no_cloud',
                                           blue_saturated = False,
                                           green_saturated = False,
                                           red_saturated = False,
                                           nir_saturated = False,
                                           swir1_saturated = False,
                                           swir2_saturated = False,
                                           contiguous = True)
            
            # Filter to remove clouds and -999 terrain issues
            ds = ds.where(cloud_free)
            ds = ds.where(ds != -999.0)
            
            # Add projection attributes
            ds.attrs['crs'] = ds.crs
            ds.attrs['affine'] = ds.affine    
            ds.attrs['geo_transform'] = ds.geobox.transform.to_gdal()
            ds.attrs['proj'] = ds.geobox.crs.wkt
            print('Masked {} with {} and filtered ' \
                  'terrain'.format(product_name, mask_product))
            
        else: 
            
            print('Did not mask {} with {}'.format(product_name, mask_product))
            
    else:
        
        print ('Did not load {}'.format(product_name)) 

    if len(ds) > 0:
        
        return ds
    
    else:
        
        return None


def rasterize_vector(input_data, cols, rows, geo_transform,
                     projection, field):
    
    """
    Rasterize a vector file and return numpy array
    :attr input_data: input shapefile path or preloaded GDAL/OGR layer
    :attr cols: width of output array in columns
    :attr rows: height of output array in rows
    :attr geo_transform: geotransform for rasterization
    :attr projection: projection for rasterization
    :attr field: shapefile field to take values from
    :returns: a 'row x col' array containg values from vector
    """
    
    # If input data is a string, import as shapefile layer
    if isinstance(input_data, str):    
    
        # Open vector with gdal
        data_source = gdal.OpenEx(vector_data_path, gdal.OF_VECTOR)
        input_data = data_source.GetLayer(0)
    
    # Set up output raster
    driver = gdal.GetDriverByName('GTiff')  # 'MEM')  # In memory dataset
    target_ds = driver.Create('test.tif', cols, rows, 1, gdal.GDT_UInt16)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)
    
    # Rasterize shapefile and extract array
    gdal.RasterizeLayer(target_ds, [1], input_data, options=["ATTRIBUTE=" + field])
    band = target_ds.GetRasterBand(1)
    out_array = band.ReadAsArray()
    target_ds = None
    
    return out_array


def write_geotiff(fname, data, geo_transform, projection, nodata_val):
    
    """
    Create a single band GeoTIFF file with data from array.
    :attr fname: output file path
    :attr data: input array
    :attr geo_transform: geotransform for output raster
    :attr projection: projection for output raster
    :attr nodata_val: value to convert to nodata in output raster
    """
    
    # Set up driver
    driver = gdal.GetDriverByName('GTiff')
    
    # Create raster of given size and projection
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Byte)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    
    # Write data to array and set nodata values
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    band.SetNoDataValue(nodata_val)
    
    # Close file
    dataset = None     
    

def tasseled_cap(sensor_data, sensor, tc_bands = ['greenness', 'brightness', 'wetness'], 
                 drop = True):
    
    """
    Computes tasseled cap wetness, greenness and brightness bands from a six
    band xarray dataset, and returns a new xarray dataset with old bands 
    optionally dropped
    :attr sensor_data: input xarray dataset with six Landsat bands
    :attr tc_bands: list of tasseled cap bands to compute 
    (valid options: 'wetness', 'greenness','brightness'
    :attr sensor: Landsat sensor used for coefficient values 
    (valid options: 'ls5', 'ls7', 'ls8')
    :attr drop: if 'drop = False', return all original Landsat bands 
    :returns: xarray dataset with newly computed tasseled cap bands
    """
    
    # Copy input dataset
    output_array = sensor_data.copy(deep = True)
 
    # Coefficients for each tasseled cap band
    wetness_coeff = {'ls5':{'blue':0.0315, 'green':0.2021, 'red':0.3102, 
                            'nir':0.1594, 'swir1':-0.6806, 'swir2':-0.6109},
                     'ls7':{'blue':0.0315, 'green':0.2021, 'red':0.3102, 
                            'nir':0.1594, 'swir1':-0.6806, 'swir2':-0.6109},
                     'ls8':{'blue':0.0315, 'green':0.2021, 'red':0.3102, 
                            'nir':0.1594, 'swir1':-0.6806, 'swir2':-0.6109}}
    
    greenness_coeff = {'ls5':{'blue':-0.1603, 'green':-0.2819, 'red':-0.4934, 
                              'nir':0.7940, 'swir1':-0.0002, 'swir2':-0.1446},
                       'ls7':{'blue':-0.1603, 'green':-0.2819, 'red':-0.4934, 
                              'nir':0.7940, 'swir1':-0.0002, 'swir2':-0.1446},
                       'ls8':{'blue':-0.1603, 'green':-0.2819, 'red':-0.4934, 
                              'nir':0.7940, 'swir1':-0.0002, 'swir2':-0.1446}}
    
    brightness_coeff = {'ls5':{'blue':0.2043, 'green':0.4158, 'red':0.5524, 
                               'nir':0.5741, 'swir1':0.3124, 'swir2':0.2303},
                        'ls7':{'blue':0.2043, 'green':0.4158, 'red':0.5524, 
                               'nir':0.5741, 'swir1':0.3124, 'swir2':0.2303},
                        'ls8':{'blue':0.2043, 'green':0.4158, 'red':0.5524, 
                               'nir':0.5741, 'swir1':0.3124, 'swir2':0.2303}}
    
    # Dict to use correct coefficients for each tasseled cap band
    analysis_coefficient = {'wetness': wetness_coeff, 
                            'greenness': greenness_coeff,
                            'brightness': brightness_coeff}
    
    # For each band, compute tasseled cap band and add to output dataset
    for tc_band in tc_bands:

        # Create xarray of coefficient values used to multiply each band of input
        coeff = xr.Dataset(analysis_coefficient[tc_band][sensor])    
        sensor_coeff = sensor_data * coeff

        # Sum all bands
        output_array[tc_band] = sensor_coeff.blue + sensor_coeff.green + \
                                sensor_coeff.red + sensor_coeff.nir + \
                                sensor_coeff.swir1 + sensor_coeff.swir2
    
    # If drop = True, remove original bands
    if drop:
        
        bands_to_drop = list(sensor_data.data_vars)        
        output_array = output_array.drop(bands_to_drop)        

    return(output_array)


def point_in_poly(lon, lat, field, shapefile):
    
    """
    Extract data from polygon that contains point
    :attr lon: x coordinate of point
    :attr lat: y coordinate of point
    :attr field: field of polygon shapefile to extract for matching points
    :attr shapefile: polygon shapefile to import
    :returns: if point falls within polygon, return field from polygon
    """

    # Convert coordinates to shapely point
    mypoint = Point(lon, lat)   

    # Extract polygon info
    with fiona.open(shapefile) as shp:

        # For each polygon, identify if point falls within polygon
        poly_idx = [poly['properties'][field] for i, poly in enumerate(shp)
                    if mypoint.within(shape(poly['geometry']))]

    # If point is within polygon
    if poly_idx:
        
        # Take first polygon to avoid multiple matches        
        return(poly_idx[0])

    # If point not found in any polygon, return none
    else: 
        
        return None


def layer_extent(layer):
    
    """
    Computes min and max extents for GDAL layer features. Compared to
    built-in ".GetExtent" that always returns unfiltered extents, this 
    allows you to compute extents of features within filtered layers 
    (e.g. layers filtered with 'SetAttributeFilter')
    """
    
    # Extract tuples of x, y, z coordinates for each point feature
    point_coords = [feature.geometry().GetPoint() for feature in layer]
    
    # Compute mins and maxes across points for each tuple element
    max_x, max_y, max_z = map(min, zip(*point_coords))
    min_x, min_y, min_z = map(max, zip(*point_coords))    
    
    return  min_x, max_x, min_y, max_y


def randomforest_train(train_shps, train_field, data_func, 
                       data_func_params = {}, classifier_params = {}):
    
    '''
    Extracts training data from xarray dataset for multiple training shapefiles
    '''

    # Output training label and pixel arrays
    training_labels_list = list()
    training_samples_list = list()

    # For each shapefile, extract datacube data using extent of points
    # and add resulting spectral data and labels to list of arrays
    for train_shp in train_shps:

        print("Importing training data from {}".format(train_shp))

        # Open vector of training points with gdal
        data_source = gdal.OpenEx(train_shp, gdal.OF_VECTOR)
        layer = data_source.GetLayer(0)      

        # Compute extents and generate spatial query
        xmin, xmax, ymin, ymax = layer_extent(layer) 
        query_train = {'x': (xmin + 500, xmax - 500),
                       'y': (ymin + 500, ymax - 500),
                       'crs': 'EPSG:3577',
                       **data_func_params}        
        print(query_train)

        # Import data  as xarray and extract projection/transform data
        training_xarray = data_func(query_train)
        geo_transform_train = training_xarray.geo_transform
        proj_train = training_xarray.proj 

        # Covert to array and rearrange dimension order
        bands_array_train = training_xarray.to_array().values
        bands_array_train = np.einsum('bxy->xyb', bands_array_train)
        rows_train, cols_train, bands_n_train = bands_array_train.shape

        # Import training data shapefiles and convert to matching raster pixels
        training_shapefile = train_shp
        training_pixels = rasterize_vector(layer, cols_train, rows_train, 
                                           geo_transform_train, proj_train, 
                                           field = train_field)  

        # Extract matching image sample data for each labelled pixel location
        is_train = np.nonzero(training_pixels)
        training_labels = training_pixels[is_train]
        training_samples = bands_array_train[is_train]

        # Remove nans from training samples
        training_labels = training_labels[~np.isnan(training_samples).any(axis=1)]
        training_samples = training_samples[~np.isnan(training_samples).any(axis=1)]

        # Append outputs
        training_labels_list.append(training_labels)
        training_samples_list.append(training_samples)

    # Combine polygon training data    
    training_labels = np.concatenate(training_labels_list, axis=0)
    training_samples = np.concatenate(training_samples_list, axis=0)  

    # Set up classifier and train on training sample data and labels
    # Options for tuning: https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/
    print("\nTraining random forest classifier...")
    classifier = RandomForestClassifier(**classifier_params) 
    classifier.fit(training_samples, training_labels)
    print("Model trained on {0} bands and "
          "{1} training samples".format(training_samples.shape[1],
                                        str(len(training_samples))))
    
    return classifier


def randomforest_classify(classifier, analysis_data, classification_output):
    
    '''
    Performs classification using xarray dataset and classifier
    '''

    geo_transform = analysis_data.geo_transform
    proj = analysis_data.proj 

    # Covert to array and rearrange dimension order
    analysis_array = analysis_data.to_array().values
    analysis_array = np.einsum('bxy->xyb', analysis_array)
    rows, cols, bands_n = analysis_array.shape
    print("Data to classify:\nRows: {0}\nColumns: {1}\nBands: {2}".format(rows, cols, bands_n))

    # Remove nodata and return flattened 'pixel x bands' array
    input_nodata = np.isnan(analysis_array).any(axis = 2)
    flat_pixels = analysis_array[~input_nodata]

    # Run classification
    print("Classification running...")
    result = classifier.predict(flat_pixels)

    # Restore 2D array by assigning flattened output to empty array
    classification = np.zeros((rows, cols))
    classification[~input_nodata] = result
    print("  Classification complete")

    # Nodata removed
    print("  " + str(np.sum(classification == 0)) + " nodata cells removed")

    # Export to file
    write_geotiff(classification_output, 
                  data = classification, 
                  geo_transform = geo_transform, 
                  projection = proj,
                  nodata_val = 0)
    print("    Classification exported")


# Set up paths
classification_output = "output_data/classification_dc_hltc_mangroves.tiff"
train_data_path = "raw_data/train"
# validation_data_path = "raw_data/test"


# Experiment with TC indices

In [2]:
def hltc_import(query):
    
    """
    Imports high and low composite data for a given spatial query, and
    return an xarray dataset with projection and geo transform attributes
    :attr query: spatial query for datacube.load()
    :returns: xarray dataset with geo transform and projection attributes
    """

    # Import data
    low_tide = dc.load(product = 'low_tide_comp_20p', **query)
    high_tide = dc.load(product = 'high_tide_comp_20p', **query)

    # Rename variables in each high/low composite so datasets can be merged
    data_vars = list(low_tide.keys())[3:]  # select only data vars, not coords
    low_tide.rename({var: "lt_" + var for var in data_vars}, inplace = True)
    high_tide.rename({var: "ht_" + var for var in data_vars}, inplace = True)

    # Combine into one dataset
    output_xarray = xr.auto_combine([low_tide, high_tide]).isel(time = 0)
    
    # Set attributes   
    output_xarray.attrs['proj'] = low_tide.geobox.crs.wkt
    output_xarray.attrs['geo_transform'] = low_tide.geobox.transform.to_gdal()    
    
    return output_xarray


# Test function for importing tasseled cap data
def tc_import(query):
    
    '''
    Wrapper around load_nbart and tasseled cap to generate data for 
    classifier training and analysis.
    :attr query: query for datacube call; for training, supply only
    non-spatial queries as spatial are generated from training data
    :returns: xarray dataset with geo transform and projection attributes
    '''
    
    nbar_example = load_nbart('ls8', query)
        
    training_xarray = tasseled_cap(sensor_data = nbar_example, 
                                   sensor = 'ls8',
                                   drop = True).median("time", keep_attrs = True)
    
    return training_xarray
        
    

# Import training data and fit model

In [3]:
# List of training files to import. Each file should cover a small enough spatial area 
# so as to not slow dc.load function excessively (e.g. 100 x 100km max)
train_shps = ["raw_data/train/training_data_tasseledcap.shp"]

classifier_params = {'n_jobs': -1,                                    
                     'n_estimators': 100,
                     'max_features': "auto",
                     'min_samples_leaf': 3,
                     'oob_score': True }

data_func_params = {'time': ('2017-03-01', '2017-06-28')}

classifier = randomforest_train(train_shps = train_shps,
                                train_field = "class",
                                data_func = tc_import, #   hltc_import,
                                data_func_params = data_func_params,
                                classifier_params = classifier_params)


Importing training data from raw_data/train/training_data_tasseledcap.shp
{'x': (352998.0302334339, 313256.0191187404), 'y': (-1317049.5107177817, -1347981.8454887671), 'crs': 'EPSG:3577', 'time': ('2017-03-01', '2017-06-28')}
Loading ls8_nbart_albers


/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: casting an xarray.Dataset to a boolean will change in xarray v0.11 to only include data variables, not coordinates. Cast the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.


Loaded ls8_nbart_albers


/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/ipykernel_launcher.py:54: FutureWarning: casting an xarray.Dataset to a boolean will change in xarray v0.11 to only include data variables, not coordinates. Cast the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.


Making mask ls8_pq_albers
Masked ls8_nbart_albers with ls8_pq_albers and filtered terrain


/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/ipykernel_launcher.py:89: FutureWarning: calling len() on an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Call len() on the Dataset.variables property instead, like ``len(ds.variables)``, to preserve existing behavior in a forwards compatible manner.



Training random forest classifier...
Model trained on 3 bands and 71 training samples


# Import analysis data and classify

In [15]:
# Set up analysis data query
lat_point, lon_point, buffer = -12.5615704994, 135.013735867, 20000
x, y = geometry.point(lon_point, lat_point, CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]
query = {'x': (x - buffer, x + buffer),
         'y': (y - buffer, y + buffer),
         'time': ('2017-03-01', '2017-06-28'),
         'crs': 'EPSG:3577'}

# Load data from datacube
analysis_xarray = tc_import(query)
# analysis_xarray = hltc_import(query)

# Run classification and export to file   
randomforest_classify(classifier = classifier,
                      analysis_data = analysis_xarray,
                      classification_output = "output_data/classification_dc_tasseledcap3.tiff")

Data to classify:
Rows: 1601
Columns: 1601
Bands: 3
Classification running...
  Classification complete
  0 nodata cells removed
    Classification exported


# Test classifier parameters

In [ ]:
# # Plot output random forest trees to file
# for n, tree_in_forest in enumerate(classifier.estimators_):

#     # Create graph and save to dot file
#     export_graphviz(tree_in_forest,
#                     out_file="figures/tree_graphs/tree.dot",
#                     feature_names = list(training_xarray.keys())[3:],
#                     class_names = ["mangrove", "water", "veg", "other"],
#                     filled=True,
#                     rounded=True)

#     # Plot as figure
#     os.system('dot -Tpng figures/tree_graphs/tree.dot -o ' + \
#               'figures/tree_graphs/tree' + str(n + 1) + '.png')

# # NOTE: Setting the `warm_start` construction parameter to `True` disables
# # support for parallelized ensembles but is necessary for tracking the OOB
# # error trajectory during training.

# # Test scenarios
# ensemble_clfs = [("max_features='sqrt'",
#                   RandomForestClassifier(warm_start=True, oob_score=True,
#                                          max_features="sqrt")),
#                  ("max_features='0.5'",
#                   RandomForestClassifier(warm_start=True, max_features=0.5,
#                                          oob_score=True)),
#                  ("max_features=None",
#                   RandomForestClassifier(warm_start=True, max_features=None,
#                                          oob_score=True))]

# ensemble_clfs = [("Leaf = 1",
#                   RandomForestClassifier(warm_start=True, 
#                                          min_samples_leaf = 1,
#                                          oob_score=True)),
#                  ("Leaf = 5",
#                   RandomForestClassifier(warm_start=True, 
#                                          min_samples_leaf = 5,
#                                          oob_score=True)),
#                  ("Leaf = 20",
#                   RandomForestClassifier(warm_start=True, 
#                                          min_samples_leaf = 20,
#                                          oob_score=True))]

# # Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
# error_rate = OrderedDict((label, []) for label, _ in ensemble_clfs)

# # Range of `n_estimators` values to explore.
# min_estimators = 1
# max_estimators = 100

# for label, clf in ensemble_clfs:
#     for i in range(min_estimators, max_estimators + 1):
#         clf.set_params(n_estimators=i)
#         clf.fit(training_samples, training_labels)

#         # Record the OOB error for each `n_estimators=i` setting.
#         oob_error = 1 - clf.oob_score_
#         error_rate[label].append((i, oob_error))

# # Generate the "OOB error rate" vs. "n_estimators" plot.
# for label, clf_err in error_rate.items():
#     xs, ys = zip(*clf_err)
#     plt.plot(xs, ys, label=label)

# plt.xlim(min_estimators, max_estimators)
# plt.xlabel("n_estimators")
# plt.ylabel("OOB error rate")
# plt.legend(loc="upper right")
# plt.show()


# Classification statistics

In [ ]:
# # Verification data
# shapefiles = glob.glob(validation_data_path + "/*.shp")
# classes = [i.split("/")[2][0:1] for i in shapefiles]
# verification_pixels = vectors_to_raster(shapefiles, rows, cols, geo_transform, proj)
# for_verification = np.nonzero(verification_pixels)
# verification_labels = verification_pixels[for_verification]
# predicted_labels = classification[for_verification]

# # Confusion matrix
# print("Confussion matrix:\n%s" %
#       metrics.confusion_matrix(verification_labels, predicted_labels))
# target_names = ['Class %s' % s for s in classes]

# # Per class report
# print("Classification report:\n%s" %
#       metrics.classification_report(verification_labels, predicted_labels,
#                                     target_names=target_names))

# # Overall classification accuracy
# print("Classification accuracy: %f" %
#       metrics.accuracy_score(verification_labels, predicted_labels))

# Old import NBAR from datacube

In [ ]:
# # Define temporal range
# start_of_epoch, end_of_epoch = '2015-01-01', '2015-03-01'

# # Define wavelengths/bands of interest
# bands_of_interest = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']


# # Define sensor of interest
# sensor1 = 'ls8'

# # Location
# lat_point, lon_point, buffer = -31.88, 152.69, 30000
# x, y = geometry.point(lon_point, lat_point, CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]

# # Set up query
# query = {'time': (start_of_epoch, end_of_epoch)}
# query['x'] = (x - buffer, x + buffer)
# query['y'] = (y - buffer, y + buffer)
# query['crs'] = 'EPSG:3577'
# print(query)

# # Group PQ by solar day to avoid idiosyncracies of N/S overlap differences in PQ algorithm performance
# pq_albers_product = dc.index.products.get_by_name(sensor1 + '_pq_albers')
# valid_bit = pq_albers_product.measurements['pixelquality']['flags_definition']['contiguous']['bits']

# # Load sensor specific band adjustment tuples for TSS 
# # ls5_tss_constant, ls5_tss_exponent = 3983, 1.6246
# # ls7_tss_constant, ls7_tss_exponent = 3983, 1.6246
# # ls8_tss_constant, ls8_tss_exponent = 3957, 1.6436   

# # Retrieve the NBAR and PQ data for sensor n
# sensor1_nbar = dc.load(product = sensor1 + '_nbar_albers', 
#                        group_by = 'solar_day', 
#                        measurements = bands_of_interest,  
#                        **query)
# sensor1_pq = dc.load(product = sensor1 + '_pq_albers', 
#                      group_by = 'solar_day', 
#                      fuse_func = pq_fuser, 
#                      **query)

# # Extract projection information
# crs = sensor1_nbar.crs.wkt
# affine = sensor1_nbar.affine

# # Ensure 1:1 match between NBAR and PQ
# sensor1_nbar = sensor1_nbar.sel(time = sensor1_pq.time)

# # Generate PQ masks and apply to remove cloud, cloud shadow and saturated observations
# s1_cloud_free = masking.make_mask(sensor1_pq,
#                                   cloud_acca = 'no_cloud',
#                                   cloud_shadow_acca = 'no_cloud_shadow',
#                                   cloud_shadow_fmask = 'no_cloud_shadow',
#                                   cloud_fmask = 'no_cloud',
#                                   blue_saturated = False,
#                                   green_saturated = False,
#                                   red_saturated = False,
#                                   nir_saturated = False,
#                                   swir1_saturated = False,
#                                   swir2_saturated = False,
#                                   contiguous = True)

# # Extract good data
# s1_good_data = s1_cloud_free.pixelquality.loc[start_of_epoch:end_of_epoch]
# sensor1_nbar = sensor1_nbar.where(s1_good_data)
# sensor1_nbar

# # Single clear timestep 6 (time 2015-02-28T23:43:05)
# raster_input = sensor1_nbar.isel(time = 6)
# raster_input.red.plot()

# # Write data to file
# write_multibandgeotiff(filename="raw_data/input_raster.tif", dataset=raster_input) 
